# LSTM Simples (Previsão da Soma das Próximas 4 Semanas)

Este notebook implementa o modelo LSTM simples para previsão da **soma das próximas 4 semanas** (previsão mensal) das taxas de morbidade respiratória em municípios brasileiros.

- **Modelo:** LSTM Simples
- **Alvo:** Soma das próximas 4 semanas (previsão mensal)
- **Input:** sequência de 12 semanas (shape: [batch, 12, 1])
- **Arquitetura:** LSTM(32, return_sequences=False) → Dense(1)
- **Perda:** MAE
- **Todo o código é modular e importado dos módulos `src/`.**

In [ ]:
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root: {project_root}")

# Add the project root to sys.path (not the src directory)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from src.preprocessing import load_city_data, prepare_data_for_model, filter_city, clean_timeseries
from src.models import build_lstm
from src.train import train_model, evaluate_model, generate_forecasts, save_predictions, save_metrics
from src.utils import plot_forecast, plot_forecast_error, plot_training_history

results_dir = os.path.join('results', 'lstm_simple')
os.makedirs(results_dir, exist_ok=True)

np.random.seed(42)

## Estrutura do Repositório

- **data/**: Um CSV por cidade, cada um com coluna de data, coluna alvo e features opcionais.
- **notebooks/**: Notebooks para cada experimento. Apenas visualização e exploração.
- **src/**: Módulos reutilizáveis:
    - `preprocessing.py`: Carregamento, normalização, split, criação de janelas
    - `models.py`: Definições de modelos (baselines, MLP, LSTM, etc.)
    - `train.py`: Rotinas de treino e avaliação
    - `utils.py`: Funções auxiliares (plot, métricas, etc.)
- **results/**: Previsões e métricas salvas.
- **instructions.md**: Guia de boas práticas.

**Toda a lógica central está em `src/`.**

## Carregamento e Exploração dos Dados

Carregue os dados de morbidade respiratória para análise. Você pode iterar sobre todas as cidades ou selecionar uma específica.

In [2]:
# Exemplo: Carregar dados de uma cidade (ajuste o caminho conforme necessário)
data_path = '../data/df_base_morb_resp.csv'
df = load_city_data(data_path)

print(f"Formato do dataset: {df.shape}")
df.head()

Formato do dataset: (6684000, 11)


,CD_MUN,week,target,PIB,DENS,URB,CO2,CH4,N2O,LAT,LON
0,1100015,0,0.515856,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
1,1100015,1,0.539765,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
2,1100015,2,0.458823,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
3,1100015,3,0.485555,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39
4,1100015,4,0.231805,3469.14,3.541043,0.000611,550.985905,92.946598,6.657747,-12.883213,-62.39


In [3]:
print("Colunas disponíveis:")
print(df.columns.tolist())
df.describe()

Colunas disponíveis:
['CD_MUN', 'week', 'target', 'PIB', 'DENS', 'URB', 'CO2', 'CH4', 'N2O', 'LAT', 'LON']


,CD_MUN,week,target,PIB,DENS,URB,CO2,CH4,N2O,LAT,LON
count,6.684000e+06,6.684000e+06,6.684000e+06,6.003816e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06,6.033600e+06
mean,3.253591e+06,5.995000e+02,4.710729e-01,1.397299e+04,8.091379e+01,1.268524e-02,3.953797e+04,7.463403e+02,1.116126e+01,-1.638302e+01,-4.637996e+01
std,9.848220e+05,3.464101e+02,3.616928e+00,1.973538e+04,3.952444e+02,4.668693e-02,1.244388e+05,1.332048e+04,2.845796e+01,8.354524e+00,6.424812e+00
min,1.100015e+06,0.000000e+00,0.000000e+00,-1.459830e+03,3.143629e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.365254e+01,-7.348400e+01
25%,2.512101e+06,2.997500e+02,0.000000e+00,4.516650e+03,1.112562e+01,1.319166e-03,4.454221e+03,6.922971e+01,4.712564e+00,-2.279264e+01,-5.102275e+01
50%,3.146280e+06,5.995000e+02,0.000000e+00,8.607535e+03,2.363456e+01,2.999792e-03,1.128696e+04,1.379897e+02,8.660524e+00,-1.785337e+01,-4.661550e+01
75%,4.119202e+06,8.992500e+02,2.773210e-01,1.684539e+04,4.817254e+01,7.148583e-03,2.817704e+04,2.405503e+02,1.352129e+01,-8.456431e+00,-4.166000e+01
max,5.300108e+06,1.199000e+03,7.359496e+02,9.208340e+05,1.344249e+04,9.768962e-01,3.137265e+06,7.075641e+05,3.683565e+03,4.685425e+00,-3.487000e+01


In [4]:
# Select city for modeling (set to None to use all cities)
CD_MUN_SELECTED = 3550308  # São Paulo

df_city = filter_city(df, cd_mun=CD_MUN_SELECTED)
df_city = clean_timeseries(df_city, target_column='target')
print(f"Selected city shape: {df_city.shape}")

Selected city shape: (1200, 11)


## Pré-processamento

Prepare os dados para o modelo LSTM simples. O input é uma sequência de 12 semanas.

In [5]:
# Preprocessamento para LSTM simples (soma das próximas 4 semanas)
model_params = {
    'sequence_length': 12,
    'forecast_horizon': 4,  # Soma das próximas 4 semanas (previsão mensal)
    'normalization': 'zscore',
    'val_size': None  # Use default (10% of train)
}

target_column = 'target'

data_dict = prepare_data_for_model(
    df=df_city,
    target_column=target_column,
    sequence_length=model_params['sequence_length'],
    forecast_horizon=model_params['forecast_horizon'],
    normalization=model_params['normalization'],
    val_size=model_params.get('val_size', None)
)

X_train = data_dict['X_train']
y_train = data_dict['y_train']
X_test = data_dict['X_test']
y_test = data_dict['y_test']
test_df = data_dict['test_df']
scaler = data_dict.get('scaler')
feature_columns = data_dict.get('feature_columns', None)

# Garantir que não ocorre erro caso X_val/y_val não existam
X_val = data_dict.get('X_val', None)
y_val = data_dict.get('y_val', None)

## Definição e Treinamento do Modelo

O modelo LSTM simples utiliza uma camada LSTM com 32 unidades e uma camada densa de saída.

In [6]:
input_shape = X_train.shape[1:]
model = build_lstm(input_shape=input_shape, units=32, loss='mae')

history = train_model(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    batch_size=32,
    epochs=100,
    patience=15,
    verbose=2
)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.1299 - val_loss: 2.2702 - learning_rate: 0.0010
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2.1299 - val_loss: 2.2702 - learning_rate: 0.0010
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7114 - val_loss: 2.2445 - learning_rate: 0.0010
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7114 - val_loss: 2.2445 - learning_rate: 0.0010
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5559 - val_loss: 2.2436 - learning_rate: 0.0010
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5559 - val_loss: 2.2436 - learning_rate: 0.0010
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6006 - val_loss: 2.2170 - learning_rate: 0.0010
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.6006 - val_loss: 2.2170 - learning_rate: 0.0010
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5053 - val_loss: 2.1754 - learning_rate: 0.0010
Epoch 6/100
32/32

## Avaliação do Modelo

Calcule MAE, RMSE e R² para o modelo LSTM simples.

In [ ]:
metrics = evaluate_model(
    model=model,
    X_test=X_test,
    y_test=y_test,
    scaler=scaler
)

print("Métricas de Avaliação:")
print(f"MAE: {metrics['mae']:.4f}")
print(f"RMSE: {metrics['rmse']:.4f}")
print(f"R²: {metrics['r2']:.4f}")

## Visualização dos Resultados

Plote os valores reais versus previstos e o erro de previsão. O alvo é a **soma das próximas 4 semanas** (previsão mensal).

In [ ]:
def reduce_to_1d(arr):
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr
    if arr.ndim == 2:
        if arr.shape[1] == 1:
            return arr.ravel()
        else:
            return arr.sum(axis=1)
    raise ValueError(f"Unexpected array shape: {arr.shape}")

# Always define robust 1D, denormalized arrays for all plots and DataFrames
y_test_1d = reduce_to_1d(y_test)
y_pred = generate_forecasts(model, X_test)
y_pred_1d = reduce_to_1d(y_pred)
if scaler is not None and hasattr(scaler, 'scale_') and hasattr(scaler, 'mean_'):
    if scaler.scale_.shape[0] > 1:
        y_test_1d = y_test_1d * scaler.scale_[0] + scaler.mean_[0]
        y_pred_1d = y_pred_1d * scaler.scale_[0] + scaler.mean_[0]
    else:
        y_test_1d = scaler.inverse_transform(y_test_1d.reshape(-1, 1)).flatten()
        y_pred_1d = scaler.inverse_transform(y_pred_1d.reshape(-1, 1)).flatten()

# Use robust 1D, denormalized arrays for all plots and DataFrames
test_dates = test_df['week'].values[-len(y_test_1d):] if 'week' in test_df.columns else np.arange(len(y_test_1d))

fig = plot_forecast(
    true_values=y_test_1d,
    predictions=y_pred_1d,
    dates=test_dates,
    title="LSTM Simples: Previsão vs Real",
    true_label="Real",
    pred_label="Previsão LSTM"
)
plt.savefig(os.path.join(results_dir, 'forecast_plot.png'))
plt.show()

fig = plot_forecast_error(
    true_values=y_test_1d,
    predictions=y_pred_1d,
    dates=test_dates,
    title="LSTM Simples: Erro de Previsão"
)
plt.savefig(os.path.join(results_dir, 'error_plot.png'))
plt.show()

plt.figure(figsize=(14,6))
plt.plot(test_df['week'], test_df['target'], label='Semanal Real', color='gray', alpha=0.5)
plt.scatter(test_df['week'].values[-len(y_pred_1d):], y_pred_1d, label='Previsto (soma 4 semanas)', color='crimson', zorder=3)
plt.scatter(test_df['week'].values[-len(y_test_1d):], y_test_1d, label='Real (soma 4 semanas)', color='royalblue', marker='x', zorder=3)
plt.title('Série Semanal Real vs. Soma Prevista das 4 Semanas (Mensal)')
plt.xlabel('Semana')
plt.ylabel('Taxa de Morbidade Respiratória')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Diagnóstico: Verificar alinhamento de y_test e y_pred
N = 10
week_idx = test_df['week'].values[-len(y_test_1d):]
df_diag = pd.DataFrame({
    'week': week_idx[:N],
    'y_test (soma 4 sem)': y_test_1d[:N],
    'y_pred (soma 4 sem)': y_pred_1d[:N]
})
display(df_diag)

# --- Advanced Evaluation and Visualization ---

# 1. Forecast vs Actual (already present)
fig1 = plot_forecast(y_test_1d, y_pred_1d, dates=test_dates, title="Forecast vs Actual")
fig1.show()

# 2. Forecast Error (already present)
fig2 = plot_forecast_error(y_test_1d, y_pred_1d, dates=test_dates, title="Forecast Error")
fig2.show()

# 3. Distribution of Errors
errors = y_pred_1d - y_test_1d
plt.figure(figsize=(10,6))
sns.histplot(errors, kde=True, bins=30, color='crimson')
plt.title('Distribution of Forecast Errors')
plt.xlabel('Error (Predicted - Actual)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# 4. Scatter Plot: Actual vs Predicted
plt.figure(figsize=(8,8))
plt.scatter(y_test_1d, y_pred_1d, alpha=0.5)
plt.plot([y_test_1d.min(), y_test_1d.max()], [y_test_1d.min(), y_test_1d.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted Scatter')
plt.grid(True)
plt.show()

# 5. Residuals Over Time
plt.figure(figsize=(12,6))
plt.plot(test_dates, errors, marker='o', linestyle='-', color='orange')
plt.axhline(0, color='black', linestyle='--')
plt.title('Residuals Over Time')
plt.xlabel('Week')
plt.ylabel('Residual (Predicted - Actual)')
plt.grid(True)
plt.show()

# 6. Rolling Mean of Errors
window = 8
rolling_error = pd.Series(errors).rolling(window=window).mean()
plt.figure(figsize=(12,6))
plt.plot(test_dates, rolling_error, color='purple')
plt.title(f'Rolling Mean of Errors (window={window})')
plt.xlabel('Week')
plt.ylabel('Rolling Mean Error')
plt.grid(True)
plt.show()

# 7. Cumulative Error
cumulative_error = np.cumsum(errors)
plt.figure(figsize=(12,6))
plt.plot(test_dates, cumulative_error, color='teal')
plt.title('Cumulative Forecast Error')
plt.xlabel('Week')
plt.ylabel('Cumulative Error')
plt.grid(True)
plt.show()

# 8. Feature Correlation Heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df_city.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Feature Correlation Heatmap')
plt.show()

# 9. Target Distribution
plt.figure(figsize=(10,6))
sns.histplot(df_city['target'], kde=True, bins=30, color='royalblue')
plt.title('Target Distribution')
plt.xlabel('Target')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# 10. Missing Values Heatmap
plt.figure(figsize=(12,4))
sns.heatmap(df_city.isnull(), cbar=False, yticklabels=False)
plt.title('Missing Values Heatmap')
plt.show()

## Salvar Resultados

Salve as previsões e métricas para comparação posterior.

In [ ]:
# Salvar previsões
preds_file = save_predictions(
    y_true=y_test_1d,
    y_pred=y_pred_1d,
    dates=test_dates,
    city_name='all',
    model_name='lstm_simple',
    output_dir=results_dir
)
print(f"Previsões salvas em: {preds_file}")

# Salvar métricas
metrics_file = save_metrics(
    metrics=metrics,
    city_name='all',
    model_name='lstm_simple',
    output_dir=results_dir,
    params=model_params
)
print(f"Métricas salvas em: {metrics_file}")

## Conclusão

O modelo LSTM simples serve como baseline neural para previsão de morbidade respiratória semanal. Compare seu desempenho com outros modelos nos próximos notebooks.